In [43]:
# #### Pre Installs
# import os
# os.environ['openAIAPIKey'] = ''
# os.environ['serpAPIKey'] = ''
!pip install google-search-results

In [33]:
appConfig = {
    "NoOfLinks" : 3,
    "oaSystemprompt" : "You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.",
    "oaUserprompt" : ""
}

In [34]:
import os

class getApiKeys:

  def __init__(self):
    pass

  def getApiKeys(self,apiKey):
    return os.environ.get(apiKey)

In [36]:
import serpapi
import json
from serpapi import GoogleSearch # Add this import statement

class searchForLinks:
  def __init__(self):
    pass

  def getSearchResults(self,searchWord):
    searpApiKey = getApiKeys()
    params = {
      "q": searchWord,
      "hl": "en",
      "gl": "us",
      "google_domain": "google.com",
      "api_key": searpApiKey.getApiKeys('serpAPIKey')
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    linksList = []

    if 'organic_results' in results:
        for res in results['organic_results']:
            if 'link' in res:  # Check if 'link' key exists in each result dictionary
                linksList.append(res['link'])
    else:
        print("No organic results found in the response.")

    return linksList[:appConfig['NoOfLinks']]

In [37]:
sfl = searchForLinks()
sfl.getSearchResults('coffee')

['https://en.wikipedia.org/wiki/Coffee',
 'https://bluebottlecoffee.com/?srsltid=AfmBOoq8gnRau5HYxQYISjN2ayTy3oODuySShGu6HG4dXRIOxORDg1dQ',
 'https://www.starbucks.com/']

In [38]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display


class webCrawler:

  def __init__(self):
    pass

  def crawlresults(self,srch):
    sfl = searchForLinks()
    linktocrawl = sfl.getSearchResults(srch)
    print(linktocrawl)
    crawlJson = {}
    for links in linktocrawl:
      crawlJson.update(self.crawl(links))

    return crawlJson

  def crawl(self, url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.title.string if soup.title else "Title not found"

    if soup.body:
      for irrelevant in soup.body(["script", "style","img","input"]):
        irrelevant.decompose()
      text = soup.body.get_text(separator="\n", strip=True)
    else:
      text = ""

    return {title : text}

    # links = [link.get('href') for link in soup.find_all('a')]
    # self.links = [link for link in links if link]

In [39]:
import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import time


class summarizer:

  def __init__(self):
    pass

  def user_prompt_for(self,title,text):
      user_prompt = f"You are looking at a website titled {title}"
      user_prompt += "The contents of this website is as follows; \
                      please provide a short summary of this website in markdown. \
                      If it includes news or announcements, then summarize these too.\n\n"
      user_prompt += text
      return user_prompt

  def messages_for(self,content):
      return [
          {"role": "system", "content": appConfig['oaSystemprompt']},
          {"role": "user", "content": content}
      ]

  def summarize(self,url):
      searpApiKey = getApiKeys()
      openai = OpenAI(api_key=searpApiKey.getApiKeys('serpAPIKey'))

      crawl = webCrawler()
      contents = crawl.crawlresults('coffee')

      for content in contents:
        user_prompt = self.user_prompt_for(content,contents[content])
        messages = self.messages_for(content)
        print(user_prompt)
      # try:
      #   response = openai.chat.completions.create(
      #       model = "gpt-4o-mini",
      #       messages = messages_for(website)
      #   )
      #   return response.choices[0].message.content
      # except openai.error.RateLimitError:
      #   print("Rate limit exceeded. Waiting for 60 seconds...")
      #   time.sleep(60)  # Wait for 60 seconds before retrying
      #   return summarize(url)





In [40]:
# summarize("coffee")
extract = summarizer()
extract.summarize('coffee')

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable